Contenido bajo licencia Creative Commons BY-NC-ND. El contenido de este Notebook es creado por Ana Beatriz Acevedo Jaramillo y Daniela González González 2020. Este material es parte del curso Análisis de Estructuras del programa de Ingeniería Civil de la Universidad EAFIT.

<img src= 
"http://www.eafit.edu.co/biblioteca/PublishingImages/logos/eafit_apa.jpg" 
         alt="GeeksforGeeks logo" 
         align="left">

# *Análisis matricial de cerchas planas*

<p style='text-align: justify;'> El siguiente programa fue desarrollado como parte del material de apoyo para realizar el análisis de cerchas planas. Debe tener en cuenta que los datos de entrada que aparecen en el programa por defecto corresponden al ejercicio explicado en el documento de word del anexo.</p>

### Sección de datos de entrada

<p style='text-align: justify;'> Esta sección contiene todas las matrices y datos que el programa utiliza para realizar el cálculo de las fuerzas internas, reacciones, etc. Estos datos pueden ser modificados dependiendo de la estructura tipo cercha que se desee analizar.</p>

In [1]:
import numpy as np

1.En la próxima celda digite el valor de cada variable teniendo en cuenta la siguiente definición:

* Nn=Número de nodos
* NE=Número de elementos
* NGL=Número de grados de libertad
* NGLL=Número de grados de libertad libres
* NGLR=Número de grados de libertad restringidos

In [2]:
Nn=4    
NE=5    
NGL=8  
NGLL=5 
NGLR=3 

2. Digite la matriz de grados de libertad tal y como se muestra en la siguiente imagen:

\begin{bmatrix}
gxi_{1} &gyi_{1} &gxf_{1} &gyf_{1}\\
gxi_{2} &gyi_{2} &gxf_{2} &gyf_{2}\\
gxi_{3} &gyi_{3} &gxf_{3} &gyf_{3}\\
gxi_{n} &gyi_{n} &gxf_{n} &gyf_{n}\end{bmatrix}


In [3]:
MGL=[[7,8,3,4],[3,4,5,6],[7,8,1,2],[1,2,5,6],[1,2,3,4]]

3. Digite las matrices de propiedades de los elementos. MA corresponde a la matriz de almacenamiento de áreas y ME a la matriz de almacenamiento de  módulos de elasticidad. Recuerde ser consistente con las unidades, para este ejercicio se usarán áreas en mm<sup>2</sup> y módulos en GPa.


\begin{bmatrix}
A_{1}\\A_{2}\\A_{3}\\A_{n}\end{bmatrix}



\begin{bmatrix}
E_{1}\\E_{2}\\E_{3}\\E_{n}\end{bmatrix}


In [4]:
MA=[750,750,1000,1000,750]
ME=[200,200,200,200,200]

4. Digite la matriz de coordenadas de la estructura tal y como se muestra en la siguiente figura. Para este ejercicio, las distancias serán ingresadas en mm.

\begin{bmatrix}
xi_{1} &yi_{1} &xf_{1} &yf_{1}\\
xi_{2} &yi_{2} &xf_{2} &yf_{2}\\
xi_{3} &yi_{3} &xf_{3} &yf_{3}\\
xi_{n} &yi_{n} &xf_{n} &yf_{n}\end{bmatrix}


In [5]:
MC=[[0,0,4000,3000],[4000,3000,8000,0],[0,0,4000,0],[4000,0,8000,0],[4000,0,4000,3000]]

5. Digite la matriz de cargas en los nodos y la matriz de desplazamiento en los grados de libertad restringidos. Para este ejercicio, se ingresaron las cargas en kN y los desplazamientos en mm.

In [6]:
F0=[[0],[-10],[0],[0],[20]]
U1=[[0],[0],[0]]

### Sección de procesamiento de datos 

<p style='text-align: justify;'> En esta sección, el programa procesa los datos ingresados en las matrices de la sección de entrada de datos para realizar los cálculos necesarios. Se sugiere no modificar ninguna línea de código, a menos que se esté seguro del cambio a realizar, ya que esto puede dañar el programa y provocar que los resultados obtenidos no sean correctos. </p>

6. Cálculo de la matriz de longitud. 

In [7]:
ML=np.zeros((NE,1))
for i in range(NE):
    ML[i,0]=np.sqrt((MC[i][2]-MC[i][0])**2+(MC[i][3]-MC[i][1])**2)
  

7. Cálculo de la matriz de rigidez local, se sugiere verificar en qué unidades queda definida esta matriz.

In [8]:
MAL=np.zeros((NE,4,4))
for i in range(NE):
    MAL[i,0,0]=(MA[i]*ME[i])/(ML[i][0])
    MAL[i,2,2]=MAL[i,0,0]
    MAL[i,0,2]=-MAL[i,0,0]
    MAL[i,2,0]=-MAL[i,0,0]

8. Cálculo de la matriz de transformación.

In [9]:
MAT=np.zeros((NE,4,4))
for i in range(NE):
    MAT[i,0,0]=(MC[i][2]-MC[i][0])/(ML[i][0])
    MAT[i,1,0]=(MC[i][3]-MC[i][1])/(ML[i][0])
    MAT[i,2,2]=MAT[i,0,0]
    MAT[i,3,2]=MAT[i,1,0]
            

9. Cálculo de la matriz de rigidez de cada elemento en coordenadas globales.

In [10]:
MAG=np.zeros((NE,4,4))
for i in range(NE):
    MAG[i]=np.dot(np.dot(MAT[i],MAL[i]),np.transpose(MAT[i]))  

10. Cálculo de la matriz de rigidez de la estructura

In [11]:
KG=np.zeros((NGL,NGL))
for i in range(NE):
    for j in range(4):
        for k in range(4):
            KG[int(MGL[i][k]-1),int(MGL[i][j]-1)]=KG[int(MGL[i][k]-1)][int(MGL[i][j]-1)]+MAG[i][j][k]

11. Subdivisión de la matriz de rigidez global de la estructura. 

In [12]:
K0 = KG[0:NGLL,0:NGLL]
K1 = KG[0:NGLL,NGLL:NGL]
K2 = KG[NGLL:NGL,0:NGLL]
K3 = KG[NGLL:NGL,NGLL:NGL]
      

12. Cálculo de reacciones y de desplazamiento en los grados de libertad libres.


In [13]:
Feff=F0-np.dot(K1,U1)
U0=np.dot((np.linalg.inv(K0)),Feff)       
F1=np.dot(K2,U0)+np.dot(K3,U1)


13.Definición de las matrices necesarias para realizar el cálculo de las fuerzas internas de cada elemento. 



In [14]:
U=np.concatenate((U0,U1))
MU=np.zeros((NE,4,1)) 
MFG=np.zeros((NE,4,1))
MFL=np.zeros((NE,4,1))
FELEM=np.zeros((NE,1))

14. Cálculo de las fuerzas internas de cada elemento.

In [15]:
for i in range(NE):
    for j in range(4):
        MU[i,j,0]=U[int(MGL[i][j])-1][0]
        
for i  in range(NE):
    MFG[i]=np.dot(MAG[i],MU[i])
       
for i  in range(NE):
    MFL[i]=np.dot(np.transpose(MAT[i]),MFG[i])

for i in range(NE):
    FELEM[i,0]=-MFL[i,0,0]

In [16]:
print(FELEM)

[[-8.33333333]
 [-8.33333333]
 [26.66666667]
 [26.66666667]
 [10.        ]]
